In [1]:
import pickle
import requests
import json
import random
from multiprocessing import Pool
import time

In [2]:
isbn2disciplineDD=pickle.load(open('../Disciplines/isbn2disciplineDD.pkl'))
isbn2disciplineLOC=pickle.load(open('../Disciplines/isbn2disciplineLOC.pkl'))
isbns=set(isbn2disciplineDD.keys()) | set(isbn2disciplineLOC.keys())
liberals=set()
with open('../BookCodes/liberals_6_22.txt') as l:
    for line in l:
        liberals.add(line.strip())
conservatives=set()
with open('../BookCodes/conservatives_6_22.txt') as c:
    for line in c:
        conservatives.add(line.strip())
isbns |= liberals
isbns |= conservatives

#### Convert using formula

In [16]:
isbns=list(isbns)
isbn13=[]
for isbn in isbns:
    check=9*1 + 7*3 + 8*1
    multiplier=3
    for d in isbn[:-1]:
        check += int(d)*multiplier
        multiplier = 1 if multiplier==3 else 3
    isbn13.append('978'+isbn[:-1]+str((10-check%10)%10))

In [18]:
random.choice(zip(isbns,isbn13))

(u'0131690256', u'9780131690257')

In [19]:
with open('../AmazonBooks/isbn2isbn13.pkl','wb') as outfile:
    pickle.dump(zip(isbns,isbn13),outfile)

#### Convert by looking online

In [94]:
def convert(isbn):
    query='request_data=%7B%22isbn%22%3A%22{}%22%7D&request_action_type=&request_code=isbn_convert'.format(isbn)
    try:
        r = requests.post('http://www.isbn.org/xmljson.php',params=query)
        converted=''.join(r.json()['results']['converted_isbn'].split('-'))
    except:
        converted='ERROR'
    time.sleep(random.random())
    return converted

In [ ]:
pool=Pool(30)
isbn13=pool.map(convert,list(isbns),len(isbns)/30)